In [10]:
import tempfile
from reportlab.lib import colors
from reportlab.pdfbase import pdfmetrics
from reportlab.pdfbase.pdfmetrics import registerFontFamily
from reportlab.pdfbase.ttfonts import TTFont
from reportlab.platypus import Paragraph, SimpleDocTemplate, Table, LongTable, Image, Spacer
from reportlab.lib.styles import getSampleStyleSheet, ParagraphStyle
from reportlab.lib.enums import TA_JUSTIFY
from io import BytesIO
from reportlab.lib.units import cm

In [11]:
import json

In [12]:
pdfmetrics.registerFont(TTFont('SimSun', 'SimSun.ttf'))  # 默认不支持中文，需要注册字体
pdfmetrics.registerFont(TTFont('SimSunBd', 'SimSun-bold.ttf'))

In [13]:
class ReportLabClass(object):
    """自动化报告模板"""

    def __init__(self):
        self.stylesheet = getSampleStyleSheet()  # 获取样式集
        parmeter = {"name": "aaa"}

        Normal = self.stylesheet['Normal']  # 获取reportlab自带样式
        BodyText = self.stylesheet['BodyText']
        Italic = self.stylesheet['Italic']
        Title = self.stylesheet['Title']
        Heading1 = self.stylesheet['Heading1']
        Heading2 = self.stylesheet['Heading2']
        Heading3 = self.stylesheet['Heading3']
        Heading4 = self.stylesheet['Heading4']
        Heading5 = self.stylesheet['Heading5']
        Heading6 = self.stylesheet['Heading6']
        Bullet = self.stylesheet['Bullet']
        Definition = self.stylesheet['Definition']
        Code = self.stylesheet['Code']

        Normal.fontName = 'SimSun'
        Italic.fontName = 'SimSun'
        BodyText.fontName = 'SimSun'
        Title.fontName = 'SimSunBd'
        Heading1.fontName = 'SimSun'
        Heading2.fontName = 'SimSun'
        Heading3.fontName = 'SimSun'
        Heading4.fontName = 'SimSun'
        Heading5.fontName = 'SimSun'
        Heading6.fontName = 'SimSun'
        Bullet.fontName = 'SimSun'
        Definition.fontName = 'SimSun'
        Code.fontName = 'SimSun'

        self.genero_paragraphStyle(**parmeter)

    def genero_paragraphStyle(self, **kwargs):
        """"""
        self.stylesheet.add(ParagraphStyle(**kwargs))


In [16]:
class ReportLabWorker(ReportLabClass):
    """自动化报告处理"""

    def __init__(self, frompath):
        """
        param frompath:源文件
        """
        super().__init__()
        self.typeDict = {"YK": "yk_process", "BY": "by_process"}
        self.frompath = frompath

    def yk_process(self):
        """云康模板"""

        # 段落样式
        self.stylesheet.add(
            ParagraphStyle(name='yk_paragraph',  # 段落
                           fontName="SimSun",
                           fontSize=10,
                           textColor='black',
                           leading=20,  # 行间距
                           spaceBefore=10,  # 段前间距
                           spaceAfter=10,  # 段后间距
                           leftIndent=0,  # 左缩进
                           rightIndent=0,  # 右缩进
                           firstLineIndent=20,  # 首行缩进，每个汉字为10
                           alignment=TA_JUSTIFY,  # 对齐方式
                           )
        )

        self.stylesheet.add(
            ParagraphStyle(name='yk_background',  # 有背景颜色的段落
                           fontName="SimSun",
                           fontSize=10,
                           textColor='black',  # 字体颜色
                           backColor='#DCDCDC',  # 背景色
                           leading=20,  # 行间距
                           spaceBefore=20,  # 段前间距
                           spaceAfter=40,  # 段后间距
                           leftIndent=0,  # 左缩进
                           rightIndent=0,  # 右缩进
                           firstLineIndent=0,  # 首行缩进，每个汉字为10
                           alignment=TA_JUSTIFY,  # 对齐方式
                           borderWidth=0.2,  # 边框粗细
                           borderColor='#A9A9A9',  # 边框颜色
                           )
        )

        self.table_style1 = [
            ('FONTNAME', (0, 0), (-1, -1), 'SimSun'),  # 字体
            ('FONTSIZE', (0, 0), (-1, -1), 10),  # 字体大小

            ('BACKGROUND', (0, 0), (0, -2), colors.gainsboro),  # 设置第一列背景颜色
            ('BACKGROUND', (1, 0), (1, -2), colors.whitesmoke),  # 设置第二列背景颜色
            ('BACKGROUND', (2, 0), (2, -2), colors.gainsboro),  # 设置第三列背景颜色
            ('BACKGROUND', (3, 0), (3, -2), colors.whitesmoke),  # 设置第四列背景颜色
            ('BACKGROUND', (0, -1), (3, -1), colors.gainsboro),  # 设置最后一行背景颜色
            ('SPAN', (0, -1), (3, -1)),  # 最后一列合并
            ('TEXTCOLOR', (0, -1), (0, -1), colors.black),  # 设置表格内文字颜色
            ('ALIGN', (0, 0), (3, 3), 'CENTER'),  # 表格左右中间对齐
            ('ALIGN', (0, -1), (-1, -1), 'RIGHT'),  # 最后一行右对齐
            ('VALIGN', (0, 0), (-1, -1), 'MIDDLE'),  # 所有表格居中对齐
            ('GRID', (0, 0), (-1, -1), 0.2, colors.white),  # 设置表格框线为白色，线宽为0.2
        ]

        style1 = self.stylesheet['yk_paragraph']
        style2 = self.stylesheet['yk_background']

        # def genero_table(self, **kwargs):

        personal_information = open(frompath, encoding='utf-8')

        information_table = personal_information.read()
        data = json.loads(information_table)

        personal_information.close

        self.story = []

        self.table_data1 = [
            [data["dataDict"]["code"]["zhName"], data["dataDict"]["code"]["value"], data["dataDict"]["name"]["zhName"],
             data["dataDict"]["name"]["value"]],
            [data["dataDict"]["sex"]["zhName"], data["dataDict"]["sex"]["value"], data["dataDict"]["age"]["zhName"],
             data["dataDict"]["age"]["value"]],
            [data["dataDict"]["sample_source"]["zhName"], data["dataDict"]["sample_source"]["value"],
             data["dataDict"]["sample_type"]["zhName"], data["dataDict"]["sample_type"]["value"]],
            [data["dataDict"]["test_method"]["zhName"], data["dataDict"]["test_method"]["value"],
             data["dataDict"]["test_organization"]["zhName"], data["dataDict"]["test_organization"]["value"]],
            [data["dataDict"]["time"]["zhName"], data["dataDict"]["time"]["value"], '', ''],
        ]

        table1 = Table(data=self.table_data1, style=self.table_style1,
                       colWidths=(2.5 * cm, 7 * cm, 2.5 * cm, 4 * cm))

        # 段落
        content1 = "<font fontname=SimSunBd>评价：</font>高于4.36%的人群——您的微生态多样性属于中等偏低水平," \
                   "提示您的肠道菌群多样性不高。肠道菌群丰富度高,是健康肠道菌群的重要特征。"

        content2 = "有益菌，也称之为益生菌，如双歧杆菌，乳酸杆菌等能合成多种人体生长发育必须的维生素，如B族维生素（维生素B1、B2、B6、B12），" \
                   "维生素K，烟酸、泛酸等，还能利用蛋白质残渣合成必需氨基酸，如天门冬氨酸、苯丙氨酸、缬氨酸和苏氨酸等，并参与糖类和蛋白质的代谢，" \
                   "同时还能促进铁、镁、锌等矿物元素的吸收。这些营养物质对人类的健康有着重要作用，一旦缺少会引起多种疾病。<br/>&nbsp&nbsp 有害菌" \
                   "，数量一旦失控大量生长，就会引发多种疾病，产生致癌物等有害物质，或者影响免疫系统的功能。<br/>&nbsp&nbsp中性菌，即具有双重作" \
                   "用的细菌，如大肠杆菌、肠球菌等，在正常情况下对健康有益，一旦增殖失控，或从肠道转移到身体其他部位，就可能引发许多问题。<br/>&nbsp&nbsp" \
                   "<font fontname=SimSunBd>评价：</font>肠道菌群处于健康的平衡状态（大肠内的益生菌数量是有害菌的1千倍到1万倍），致病菌或者条件" \
                   "致病菌以很少的数目存在，它们产生的有毒代谢物不足以对人体的健康产生危害。"

        content3 = "<br/>&nbsp&nbsp ▲ 尽快调整生活习惯,改善自身肠道健康<br/>&nbsp&nbsp ▲ 保持一定的锻炼频率<br/>&nbsp&nbsp ▲ 多选择粗杂粮和含糖少的蔬菜," \
                   "蛋白质来源可选择大豆及豆制品<br/>&nbsp&nbsp ▲ 减少钠盐的摄入,合理调整饮食结构,戒烟及限制饮酒减少饱和脂肪酸和胆固醇的摄入"

        content4 = "*注意:本报告仅作为健康研究结果供参考,不作为临床诊断依据。本检测仅对本样品负责。"

        story.append(Paragraph("肠道菌群检测报告", self.Title))
        story.append(table1)
        story.append(Spacer(240, 20))  # 添加空白行
        story.append(Paragraph(content1, style1))
        story.append(Spacer(240, 20))
        story.append(Paragraph(content2, style1))
        story.append(Paragraph(content3, style2))
        story.append(Paragraph(content4, style1))
        
        
        
        
        parameter = {}
        self.genero_paragraphStyle(parameter)
        pass

    def process(self, typeName):
        """
        :type: 模板类型
        :return: doc
        """
        type_process = self.typeDict.get(typeName)
        if type_process:
            raise ValueError("该模板不存在")
        process = getattr(self, type_process)
        return process()

    def run(self, file_path):
        """启动化报告"""
        pass


In [20]:
if __name__ == '__main__':
    frompath = (r"C:\Users\chen\Desktop\data.json")
    rpc = ReportLabWorker(frompath)
    doc = SimpleDocTemplate('111.pdf')
    file_path = doc.build(self.story)
    rpc.run(file_path)

NameError: name 'self' is not defined

In [5]:
personal_information = open(r'C:\Users\chen\Desktop\data.json',encoding='utf-8')

information_table = personal_information.read()
data = json.loads(information_table)

personal_information.close

story = []

In [6]:
table_data1 = [[data["dataDict"]["code"]["zhName"], data["dataDict"]["code"]["value"], data["dataDict"]["name"]["zhName"],data["dataDict"]["name"]["value"]],
            [data["dataDict"]["sex"]["zhName"],data["dataDict"]["sex"]["value"], data["dataDict"]["age"]["zhName"], data["dataDict"]["age"]["value"]],
            [data["dataDict"]["sample_source"]["zhName"], data["dataDict"]["sample_source"]["value"],data["dataDict"]["sample_type"]["zhName"], data["dataDict"]["sample_type"]["value"]],
            [data["dataDict"]["test_method"]["zhName"], data["dataDict"]["test_method"]["value"], data["dataDict"]["test_organization"]["zhName"],data["dataDict"]["test_organization"]["value"]],
            [data["dataDict"]["time"]["zhName"],data["dataDict"]["time"]["value"] , '', ''],
            ]

In [7]:
table_data1

[['编号:', 'xxxxxxx', '姓名:', '小红'],
 ['性别:', '女', '年龄:', '20'],
 ['样本来源:', 'XX省XX市XX区XX街道XX社区居委会', '样本类型:', '粪便'],
 ['检测方法:', '下一代高通量测序技术 \n（Illumina）', '检测机构:', 'XX医院检验医学部'],
 ['检测时间:2022.2', '2022.2', '', '']]

In [8]:
table_style1 = [
    ('FONTNAME', (0, 0), (-1, -1), 'SimSun'),  # 字体
    ('FONTSIZE', (0, 0), (-1, -1), 10),  # 字体大小

    ('BACKGROUND', (0, 0), (0, -2), colors.gainsboro),  # 设置第一列背景颜色
    ('BACKGROUND', (1, 0), (1, -2), colors.whitesmoke),  # 设置第二列背景颜色
    ('BACKGROUND', (2, 0), (2, -2), colors.gainsboro),  # 设置第三列背景颜色
    ('BACKGROUND', (3, 0), (3, -2), colors.whitesmoke),  # 设置第四列背景颜色
    ('BACKGROUND', (0, -1), (3, -1), colors.gainsboro),  # 设置最后一行背景颜色
    ('SPAN', (0, -1),(3, -1)),  # 最后一列合并
    ('TEXTCOLOR', (0, -1), (0, -1), colors.black),  # 设置表格内文字颜色
    ('ALIGN', (0, 0), (3, 3), 'CENTER'),  # 表格左右中间对齐
    ('ALIGN', (0, -1), (-1, -1), 'RIGHT'),  # 最后一行右对齐
    ('VALIGN', (0, 0), (-1, -1), 'MIDDLE'),  # 所有表格居中对齐
    ('GRID', (0, 0), (-1, -1), 0.2, colors.white),  # 设置表格框线为白色，线宽为0.2
    ]

In [9]:
table1 = Table(data=table_data1, style=table_style1, colWidths= (2.5*cm , 7*cm , 2.5*cm , 4*cm))

In [10]:
story.append(table1)

In [11]:
story

[Table(
  rowHeights=[None, None, None, None, None],
  colWidths=(70.86614173228347, 198.4251968503937, 70.86614173228347, 113.38582677165354),
 [['编号:', 'xxxxxxx', '姓名:', '小红'],
    ['性别:', '女', '年龄:', '20'],
    ['样本来源:', 'XX省XX市XX区XX街道XX社区居委会', '样本类型:', '粪便'],
    ['检测方法:', '下一代高通量测序技术 \n（Illumina）', '检测机构:', 'XX医院检验医学部'],
    ['检测时间:2022.2', '2022.2', '', '']]
 ) # end table]

In [12]:
doc = SimpleDocTemplate('333.pdf')

In [13]:
doc.build(story)